# DL Method to find similar movies (CBOW)

In [3]:
# from datasets import load_dataset
# dataset = load_dataset("ashraq/movielens_ratings")
# print(dataset)

# train_data = dataset['train']
# val_data = dataset['validation']
# len(set(train_data['movieId'])) #>15,000

# print(dataset)



Generating train split:   0%|          | 0/891382 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99043 [00:00<?, ? examples/s]

In [10]:
# train_data = train_data.map(lambda x: {'imdbId': x['imdbId'], 'tmdbId': x['tmdbId'], 'movie_id': x['movie_id'], 'user_id': x['user_id'], 'rating': x['rating'], 'title': x['title'], 'genres': x['genres'], 'posters': x['posters']}) 

# # save train_data to a csv file
# train_data.to_csv('train_data.csv')

# val_data = val_data.map(lambda x: {'imdbId': x['imdbId'], 'tmdbId': x['tmdbId'], 'movie_id': x['movie_id'], 'user_id': x['user_id'], 'rating': x['rating'], 'title': x['title'], 'genres': x['genres'], 'posters': x['posters']}) 

# val_data.to_csv('val_data.csv')

DatasetDict({
    train: Dataset({
        features: ['imdbId', 'tmdbId', 'movie_id', 'user_id', 'rating', 'title', 'genres', 'posters'],
        num_rows: 891382
    })
    validation: Dataset({
        features: ['imdbId', 'tmdbId', 'movie_id', 'user_id', 'rating', 'title', 'genres', 'posters'],
        num_rows: 99043
    })
})


--- BEGIN ---

In [16]:
# !pip install gensim

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB 991.0 kB/s eta 0:00:25
   ---------------------------------------- 0.3/24.0 MB 3.3 MB/s eta 0:00:08
   - -------------------------------------- 0.7/24.0 MB 5.6 MB/s eta 0:00:05
   - -------------------------------------- 1.1/24.0 MB 7.3 MB/s eta 0:00:04
   -- ------------------------------------- 1.6/24.0 MB 7.9 MB/s eta 0:00:03
   --- ------------------------------------ 2.1/24.0 MB 8.8 MB/s eta 0:00:03
   ---- ----------------------------------- 2.8/24.0 MB 9.4 MB/s eta 0:00:03
   ----- ---------------------------------- 3.4/24.0 MB 9.7 MB/s eta 0:00:03
   ------ --------------------------------- 3.9/24.0 MB 10.0 MB/s eta 0:00:03
   ------- -------------------------------- 4.4/24.0 MB 10.0 MB/s eta 0:00:02
   -------- ------------------------------- 4.9/24.0 MB 10.2 MB/s eta 0:00:02
   --------- ------------------------------ 5.4/24.0 MB 10.2 MB/s eta 0:00:02


In [24]:
# Import(ant) stuff 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import gensim.downloader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm

In [36]:
df_train = pd.read_csv('train_data.csv')
df_val = pd.read_csv('val_data.csv')
df_users = pd.read_csv('biased_data.csv')

In [32]:
# Count the number of unique uids in the training data
print(df_train['tmdbId'].nunique())  #> 43584


# Extract the tmdbId and title from the training data and store it in another csv
df_train[['tmdbId', 'title', 'genres']].to_csv('tmdbId_title.csv')

df_tmdb = pd.read_csv('tmdbId_title.csv')
df_tmdb = df_tmdb.dropna()
df_tmdb['tmdbId'] = df_tmdb['tmdbId'].astype(int)
# Drop the duplicate rows
df_tmdb = df_tmdb.drop_duplicates(subset='tmdbId', keep='first')
# Drop the rows where the genre is (no genres listed)
df_tmdb = df_tmdb[df_tmdb['genres'] != '(no genres listed)']

df_tmdb.to_csv('tmdbId_title.csv', index=False)

df_tmdb = df_tmdb.assign(title = pd.NA, overview = pd.NA)

df_tmdb.head()





15254


,Unnamed: 0,tmdbId,title,genres,overview
0,0,150540,<NA>,Adventure|Animation|Children|Comedy|Drama|Fantasy,<NA>
1,1,393732,<NA>,Thriller,<NA>
2,2,271110,<NA>,Action|Sci-Fi|Thriller,<NA>
3,3,278927,<NA>,Adventure|Drama|Fantasy,<NA>
4,4,440021,<NA>,Horror|Mystery|Thriller,<NA>


<!--  -->

In [54]:
# Ask the user what type of movies they want to watch? 
u_genre = ['Comedy']


def filter_user_genre(df, u_genre):
    filtered_df = df[df['genres'].str.contains('|'.join(u_genre), case=False, na=False)]
    return filtered_df

u_fil = filter_user_genre(df_tmdb, u_genre)
print(u_fil.shape)

u_fil.to_csv('filtered_tmdbId_title.csv', index=False)


(3657, 5)


In [47]:
# # Function to find the highest-rated movie with specified genres and type 'movie'
# def find_highest_rated_movie(df, genres):
#     # Split the user input into a list of genres
#     genre_list = [genre.strip() for genre in genres.split(',')]
    
#     # Step 1: Filter by genres (AND join)
#     filtered_df = df
#     for genre in genre_list:
#         filtered_df = filtered_df[filtered_df['Genres'].str.contains(genre, case=False, na=False)]
    
#     # Step 2: Filter by title type 'movie'
#     filtered_df = filtered_df[filtered_df['Title Type'].str.lower() == 'movie']
    
#     # Step 3: Check if filtered dataframe is not empty
#     if filtered_df.empty:
#         print("No movies found matching the specified genres.")
#         return None
    
#     # Step 4: Find the highest IMDb rated movie
#     highest_rated_movie = filtered_df[filtered_df['IMDb Rating'] == filtered_df['IMDb Rating'].max()]
    
#     return highest_rated_movie

# highest_rated_movie = find_highest_rated_movie(df_users, 'Drama, Comedy')

# print(highest_rated_movie)



        Const  Your Rating  Date Rated       Title Original Title  \
12  tt2096673            8  2024-06-03  Inside Out     Inside Out   

                                     URL Title Type  IMDb Rating  \
12  https://www.imdb.com/title/tt2096673      Movie          8.1   

    Runtime (mins)  Year                                             Genres  \
12            95.0  2015  Animation, Comedy, Family, Adventure, Drama, F...   

    Num Votes Release Date                      Directors  
12     789828   2015-07-24  Pete Docter,Ronnie Del Carmen  
Empty DataFrame
Columns: [Unnamed: 0, tmdbId, title, genres, overview]
Index: []


In [51]:
# # check if the movie is in tmdb data -- else add the movie to the tmdb data (and find the overview using the tmdb api)
# def check_movie_in_tmdb(df, title):
#     # For all the "title" values in the dataframe, check if the input title is a substring
#     movie_in_tmdb = df['title'].str.contains(title, case=False, na=False)
#     return movie_in_tmdb

# if_movie_in_tmdb = check_movie_in_tmdb(df_tmdb, highest_rated_movie['Title'].values[0])
# # return the index of the true value
# if_movie_in_tmdb = if_movie_in_tmdb[if_movie_in_tmdb].index

# print(if_movie_in_tmdb)

Index([], dtype='int64')


In [55]:
# Making API calls to get the overview of the movies -- filter this based on the user input otherwise it will take years
for index, row in tqdm(u_fil.iterrows()):
    headers = {
        "accept": 'application/json',
        "Authorization": "eb4413659c218290f7375c1651fb7f5e"
    }
    url = "https://api.themoviedb.org/3/movie/" + str(row['tmdbId']) + "?api_key=eb4413659c218290f7375c1651fb7f5e"

    response = requests.get(url, headers=headers)
    result = response.json()

    try:
        u_fil.loc[index, 'title'] = result['title']
        u_fil.loc[index, 'overview'] = result['overview']
    except:
        continue

df = u_fil.dropna()
df.to_csv('overview_data.csv')

0it [00:00, ?it/s]

Using the tmdbId we get the overview of the movie(s).
Drop the duplicate rows 

In [ ]:
# Save the model to disk 

In [61]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean(text):
    # Remove HTML tags and patterns
    clean_text = BeautifulSoup(text, "html.parser").get_text()

    # Convert text to lowercase
    clean_text = clean_text.lower()

    # Tokenize the text and remove stopwords
    words = nltk.word_tokenize(clean_text)
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    words = [lemmatizer.lemmatize(word) for word in words]

    # Remove punctuation and non-alphanumeric characters
    words = [word for word in words if re.match(r'^[a-zA-Z0-9]+$', word)]

    # Join the words back into a cleaned text
    cleaned_text = ' '.join(words)

    return cleaned_text

df['clean_overview'] = df['overview'].apply(clean)
df = df.drop(['overview'], axis=1)
df.to_csv('cleaned_overview_data.csv')

C:\Users\jacob\AppData\Local\Temp\ipykernel_4144\54270235.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  clean_text = BeautifulSoup(text, "html.parser").get_text()
C:\Users\jacob\AppData\Local\Temp\ipykernel_4144\54270235.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_overview'] = df['overview'].apply(clean)


In [62]:
pretrained_model = gensim.downloader.load('word2vec-google-news-300')




[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [63]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df = 2, stop_words='english')
tfidf_vectors = tfidf.fit_transform(df['clean_overview'])
tfidf_feature = tfidf.get_feature_names_out()
vectors_df = pd.DataFrame(tfidf_vectors.toarray(), columns = tfidf_feature)



In [66]:
from nltk.tokenize import word_tokenize

# Making a corpus 
corpus = df['clean_overview'].apply(word_tokenize).tolist()

print(corpus)


[['adventurous', 'balloon', 'ride', 'cloud', 'metropolis', 'academy', 'director', 'pete', 'docter', 'monster', 'taken', 'audience', 'unique', 'imaginative', 'place', 'disney', 'pixar', 'original', 'movie', 'inside', 'take', 'u', 'extraordinary', 'location', 'mind'], ['special', 'bond', 'develops', 'inflatable', 'robot', 'baymax', 'prodigy', 'hiro', 'hamada', 'team', 'group', 'friend', 'form', 'band', 'hero'], ['based', 'true', 'story', 'two', 'young', 'men', 'david', 'packouz', 'efraim', 'diveroli', '300', 'million', 'contract', 'pentagon', 'arm', 'america', 'ally', 'afghanistan'], ['chef', 'carl', 'casper', 'suddenly', 'quits', 'job', 'prominent', 'los', 'angeles', 'restaurant', 'refusing', 'compromise', 'creative', 'integrity', 'controlling', 'owner', 'left', 'figure', 'next', 'finding', 'miami', 'team', 'friend', 'son', 'launch', 'food', 'truck', 'taking', 'road', 'chef', 'carl', 'go', 'back', 'root', 'reignite', 'passion', 'kitchen', 'zest', 'life', 'love'], ['comedy', 'follows', '

In [67]:
doc_vectors = []
for index, desc in tqdm(enumerate(corpus)):
    weighted_word_vector = np.zeros(300)
    weighted_sum = 0
    for word in desc:
        if word in tfidf_feature and word in pretrained_model:
            weighted_word_vector += pretrained_model[word] * vectors_df.loc[index, word]
            weighted_sum += vectors_df.loc[index, word]
        if weighted_sum != 0:
            weighted_word_vector /= weighted_sum
    doc_vectors.append(weighted_word_vector)

0it [00:00, ?it/s]

In [68]:
cosine_similarities = cosine_similarity(doc_vectors, doc_vectors)


In [69]:
def recommendation(title):
    indices = pd.Series(df.index, index = df['title']).drop_duplicates()
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    print(sim_scores)
    book_indices = [i[0] for i in sim_scores]
    recommend = df.iloc[book_indices]["title"]
    return recommend

In [70]:
recommendation("Inside Out")

[(916, 0.9412959555235175), (568, 0.7044504539775304), (2499, 0.5303163373466375), (3172, 0.4537291664411333), (643, 0.44422786700489525)]


25428      Have Fun, Vasya!
9764                Genesis
234214          Dry Martina
508809           Between Us
12511     Where Am I Going?
Name: title, dtype: object